# Semantic Kernel Exercise and Tutorial in C#
## Data Science Solutions on Microsoft Azure, 2nd Edition
## Julian Soh and Priyanshi Singh

In [10]:
//Define variables to hold API endpoint, deployment name, and api key
var azureEndpoint = "https://jsopenai-east.openai.azure.com/";
var model = "gpt-35";
var apiKey = "75b6983f87db4926ab950aa947fba9c3";

In [17]:
using System;
using System.ComponentModel;
using System.Collections.Generic;
using System.IO;
using System.Text.Json;
using System.Threading.Tasks;

Error: (2,7): error CS0246: The type or namespace name 'system' could not be found (are you missing a using directive or an assembly reference?)

Add nuGet package for Microsoft Semantic Kernel

In [12]:
#r "nuget: Microsoft.SemanticKernel, 1.15.0"

Installed Packages Microsoft.SemanticKernel, 1.15.0

Load libraries and instantiate the kernel

In [18]:
using Microsoft.SemanticKernel;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.Logging.Abstractions;
using Microsoft.Extensions.DependencyInjection;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// Inject your logger 
// see Microsoft.Extensions.Logging.ILogger @ https://learn.microsoft.com/dotnet/core/extensions/logging
ILoggerFactory myLoggerFactory = NullLoggerFactory.Instance;

var builder = Kernel.CreateBuilder();
builder.Services.AddSingleton(myLoggerFactory);

var kernel = builder.Build();

Define the Endpoint, keys, and model deployment name

Test create an instance of the kernel and configure with values above

In [19]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);

var kernel = builder.Build();

Import the email plugin from file by importing the plugin from ~\prompt_template_samples\EmailPlugin\MarketingMail

In [20]:
// MarketingEmail directory path
var MarketingEmailPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "prompt_template_samples", "EmailPlugin");
var InventoryPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "prompt_template_samples", "InventoryPlugins");

// Load the MarketingEmailplugin from the Plugins Directory
var MarketingEmailPluginFunctions = kernel.ImportPluginFromPromptDirectory(MarketingEmailPluginDirectoryPath);
var InventoryPluginFunctions = kernel.ImportPluginFromPromptDirectory(InventoryPluginDirectoryPath);

In [21]:
// 2. Enable automatic function calling
OpenAIPromptExecutionSettings openAIPromptExecutionSettings = new() 
{
    ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions
};

Using the Plugin to generate the marketing email for cycling gear

In [7]:
// Construct arguments
var arguments = new KernelArguments() { ["input"] = "Cycling" , ["recipient"] = "Julian Soh"};

// Run the Function called Joke
var result = await kernel.InvokeAsync(MarketingEmailPluginFunctions["MarketingMail"], arguments);

// Return the result to the Notebook
Console.WriteLine(result);

Subject: Don't Miss Out on Our Special Sales Event - Up to 50% Off on Cycling Merchandise

Dear Julian,

We hope this email finds you well. We are excited to announce a special sales event on our cycling merchandise, with discounts of up to 50% off!

Whether you're a seasoned cyclist or just starting out, we have everything you need to enhance your cycling experience. From high-quality bikes to comfortable apparel and accessories, we've got you covered.

Here are some of the amazing deals you can expect:

- 50% off on select bikes
- 40% off on cycling jerseys and shorts
- 30% off on helmets and gloves
- 20% off on cycling shoes and pedals

This is a limited-time offer, so don't miss out on the chance to upgrade your cycling gear at unbeatable prices. Visit our website or drop by our store to take advantage of these amazing deals.

Please note that this email is being sent to you because you opted into our mailing list. If you no longer wish to receive emails from us, you can unsubscrib

In [27]:
var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();
var history = new ChatHistory();

string? userInput;
do {
    // Collect user input
    //Console.Write("User > ");
    //userInput = Console.ReadLine();
    userInput = "I want to buy cycling bibs";

    // Add user input
    history.AddUserMessage(userInput);

    // 3. Get the response from the AI with automatic function calling
    var result = await chatCompletionService.GetChatMessageContentAsync(
        history,
        executionSettings: openAIPromptExecutionSettings,
        kernel: kernel);

    // Print the results
    Console.WriteLine("Assistant > " + result);

    // Add the message from the agent to the chat history
    history.AddMessage(result.Role, result.Content ?? string.Empty);
} while (userInput is not null);

Assistant > Great! Do you need help finding a place to purchase them or do you have a specific store in mind already?
Assistant > Okay, here are few popular online stores where you can purchase cycling bibs:

1. Competitive Cyclist
2. Wiggle
3. Chain Reaction Cycles
4. Jenson USA
5. Amazon

You can also check with your local bike shop to see if they carry any cycling bibs or if they can special order them for you.
Assistant > Sure, would you like some recommendations on specific cycling bibs for men or women? And what is your budget for this purchase?


Error: Command cancelled.